# Guild AI Interactive Python Quick Start

This Notebook mirrors the steps in [Guild AI Quick Start](https://guild.ai/docs/start/):

- Part 1 - Experiment basics
    - Create a mock training function
    - Generate a run with Guild
    - Examine the run
    - Generate a second run
    - Compare runs
- Part 2 - Hyperparameter optimization

It features the module `guild.ipy`, which is a Python interface that can be used interactively in Notebooks or by your own Python code. 

## Notebooks config

Modify the variables below to change Notebook configuration.

`GULID_HOME` is the location of generated runs. By default it is the location subdirectory "guild-env". Note that we initialize this directory below by permanently deleting any runs it contains before proceeding. If you don't want to delete runs in `GUILD_HOME`, set `DELETE_RUNS_ON_INIT` to `False` below.

In [1]:
GUILD_HOME = "guild-env"

`DELETE_RUNS_ON_INIT` determines whether or not runs are initially deleted from `GUILD_HOME` below. As this Notebook is for demonstration purposes, it's usually a good idea to delete any existing runs before proceeding. To prevent any runs from being deleted, set `DELETE_RUNS_ON_INIT` to `False`.

In [2]:
DELETE_RUNS_ON_INIT = True

## Guild interactive Python API

Guild AI functionality is available through a Notebook compatible interface defined in `guild.ipy`.

Import this module as `guild`:

In [3]:
import guild.ipy as guild

## Initialize Guild Home

Guild home is a directory containing the runs that Guild generates. This Notebook uses the directory defined by `GUILD_HOME`. Ensure that the directory exists and use `set_guild_home()` to set the value. 

In [4]:
import os

if not os.path.exists(GUILD_HOME):
    os.mkdir("guild-env")
    
guild.set_guild_home("guild-env")

Clear the director of any runs.

In [31]:
if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))

Deleted 1 run(s)


## Mock training function

Create a mock training script. This function doesn’t actually train anything, but simulates the training process of accepting hyperparameters as inputs and generating a loss.

In [32]:
import numpy as np

def train(x=0.1, noise=0.1):
    loss = (np.sin(5 * x) * (1 - np.tanh(x ** 2)) + np.random.randn() * noise)
    print("loss: %f" % loss)

Run the mock training function a couple times to see how it works.

**NOTE:** The function uses a random component to simulate training "noise". This causes the results to differ across runs.

In [33]:
print("Trial 1:")
train(x=-2.0)

Trial 1:
loss: 0.180942


In [34]:
print("Trial 2:")
train(x=0.0)

Trial 2:
loss: 0.037488


In [35]:
print("Trial 3:")
train(x=2.0)

Trial 3:
loss: 0.202648


## Generate a run

Run the mock train function using Guild:

In [36]:
run, return_val = guild.run(train)

loss: 0.574138


The `run()` function returns a tuple of the Guild run and the return value of the function.

In this case, our mock training function doesn't return a value.

In [37]:
print(return_val)

None


The `run` variable is a Python object that represents the run.

## Examine the run

We can work with the run object directly to get information about it.

Each run has a uniqiue ID:

In [38]:
run.id

'658428347af411e9b40fe4a471939b0d'

Each run is associated with a unique directory, which includes the ID. The run directory contains metadata and output associated with the run.

In [39]:
run.dir

'guild-env/runs/658428347af411e9b40fe4a471939b0d'

Our mock training script doesn't generate any files, so the run directory is empty, with the exception of a `.guild` subdirectory.

In [40]:
os.listdir(run.dir)

['.guild']

The `.guild` subdirectory contains run output and metadata. Outputs include the output printed to the script during the operation (`output`) and any scalars logged (`events.*`). Guild logs scalar output using TensorFlow event files, which can be read using TensorBoard.

In [41]:
os.listdir(os.path.join(run.dir, ".guild"))

['output', 'opref', 'events.out.tfevents.1558352570.omaha', 'attrs']

Run metadata includes *attributes*. You can list the attribute names using `attr_names()`:

In [42]:
run.attr_names()

['exit_status', 'flags', 'id', 'initialized', 'started', 'stopped']

Read an attribute value using `get()`. For example, to read the *flags* attribute, use:

In [43]:
run.get("flags")

{'noise': 0.1, 'x': 0.1}

Flags are values provided to the function. In this case, both values are defined as function keyword default values. Later you run `train` using explicit flag values.

Each run has a *status*, which indicates if the run is still running and whether or not it completed successfully or terminated with an error.

In [44]:
run.status

'completed'

## List runs

List runs using `guild.runs()`:

In [45]:
runs = guild.runs()
runs

,run,operation,started,status,label
0,65842834,train(),2019-05-20 06:42:50,completed,


Each time you run an operation, the run appears in the data frame generated by `runs()`.

## Print run info

From `runs`, we can print information for the latest run using `runs.info()`.

In [46]:
runs.info()

id: 658428347af411e9b40fe4a471939b0d
operation: train()
status: completed
started: 2019-05-20 06:42:50
stopped: 2019-05-20 06:42:50
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/658428347af411e9b40fe4a471939b0d
flags:
  noise: 0.1
  x: 0.1


This information reflects the information you saw in the prior section.

## Run scalars

Run results are recorded as *scalars*. You can list scalars in two ways.

First, you can specify the `scalars` flag to `info()`:

In [47]:
runs.info(scalars=True)

id: 658428347af411e9b40fe4a471939b0d
operation: train()
status: completed
started: 2019-05-20 06:42:50
stopped: 2019-05-20 06:42:50
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/658428347af411e9b40fe4a471939b0d
flags:
  noise: 0.1
  x: 0.1
scalars:
  loss: 0.574138 (step 0)


Second, you can get a data frame containing scalars using the `scalars()` method.

In [48]:
scalars = runs.scalars()
scalars

,avg_val,count,first_step,first_val,last_step,last_val,max_step,max_val,min_step,min_val,prefix,run,tag,total
0,0.574138,1,0,0.574138,0,0.574138,0,0.574138,0,0.574138,,658428347af411e9b40fe4a471939b0d,loss,0.574138


Guild stores aggregates of each tag, including *first*, *last*, *max*, *min*, and *average*.

You can use the various facilities in data frame to get scalar values.

In [49]:
scalars.query("(run == '%s') and (tag == 'loss')" % run.id)["last_val"]

0    0.574138
Name: last_val, dtype: float64

## Generate a second run

Run train again with different flags along with a run *label*:

In [50]:
_ = guild.run(train, x=0.2, _label="run 2")

loss: 0.893944


List runs:

In [51]:
guild.runs()

,run,operation,started,status,label
0,7c5a8f1c,train(),2019-05-20 06:43:28,completed,run 2
1,65842834,train(),2019-05-20 06:42:50,completed,


Note that the latest run contains the label "run 2" as specified in the `run()` call.

## Compare runs

Use `compare()` to generate a data frame containing both flags and scalars.

In [52]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,7c5a8f1c,train(),2019-05-20 06:43:28,0:00:00,completed,run 2,0.1,0.2,0,0.893944
1,65842834,train(),2019-05-20 06:42:50,0:00:00,completed,,0.1,0.1,0,0.574138


## Grid search

Grid search &mdash; also referred to as a parameter sweep &mdash; is a form of hyperparameter tuning that uses exhaustive search over a manually defined set of hyperparameter values.

To perform a grid search in Guild, provide a list of values to use for any given flag. If you specify lists for multiple flags, Guild runs trials for each possible flag value combination.

Run `train()` using a range of values:

In [53]:
_ = guild.run(train, x=[-0.5,-0.4,-0.3,-0.2,-0.1])

Running train (noise=0.1, x=-0.5):
loss: -0.449091
Running train (noise=0.1, x=-0.4):
loss: -0.532630
Running train (noise=0.1, x=-0.3):
loss: -0.854539
Running train (noise=0.1, x=-0.2):
loss: -0.738916
Running train (noise=0.1, x=-0.1):
loss: -0.578615


This command generates five trials, one for each specified value of `x`.

Use `runs().compare()[:5]` to compare the last five results:

In [54]:
guild.runs().compare()[:5]

,run,operation,started,time,status,label,noise,x,step,loss
0,80a7905a,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.1,0,-0.578615
1,80a79059,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.2,0,-0.738916
2,80a79058,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.3,0,-0.854539
3,80a79057,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.4,0,-0.532630
4,80a79056,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.5,0,-0.449091


View the top-three results by loss:

In [55]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
2,80a79058,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.3,0,-0.854539
1,80a79059,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.2,0,-0.738916
0,80a7905a,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.1,0,-0.578615


Based on our mock training function, the "best" result (i.e. the run with the lowest *loss*) should be the run where `x` is close to `-0.3`. Because there's a random component (i.e. the `noise` parameter) your results may show best results with different values for `x`.

Below is an image that plots *loss* for values of *x*, showing the lowest loss where x is approximately `-0.3`.

<img src="bayesian-optimization.png" style="margin-left:0">

[Image credit: *Bayesian optimization with skopt*](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

## Random search

Random search is a method used in machine learning to explore hyperparameter spaces at random.

To run a series of runs using random values over a specified range, use the `slice` function to specify a range for a flag. To specify the number of trials, specify `_max_trials`. The default number of trials is `20`.

Run train five times with random values of `x` over the range `-2.0` to `2.0`.

In [56]:
_ = guild.run(train, x=slice(-2.0,2.0), _max_trials=5)

Running train (noise=0.1, x=1.7770005973382155):
loss: 0.066694
Running train (noise=0.1, x=-0.015538373916255566):
loss: -0.054769
Running train (noise=0.1, x=-0.019494672607218266):
loss: -0.316245
Running train (noise=0.1, x=-0.5594065062475779):
loss: -0.317656
Running train (noise=0.1, x=0.5916846182595901):
loss: 0.240205


Compare the results again.

In [57]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,87f6c574,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,0.591685,0,0.240205
1,87f6c573,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,-0.559407,0,-0.317656
2,87f6c572,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,-0.019495,0,-0.316245
3,87f6c571,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,-0.015538,0,-0.054769
4,87f6c570,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,1.777001,0,0.066694
5,80a7905a,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.100000,0,-0.578615
6,80a79059,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.200000,0,-0.738916
7,80a79058,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.300000,0,-0.854539
8,80a79057,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.400000,0,-0.532630
9,80a79056,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.500000,0,-0.449091


List the three runs with the lowest loss:

In [58]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
7,80a79058,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.3,0,-0.854539
6,80a79059,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.2,0,-0.738916
5,80a7905a,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.1,0,-0.578615


## Bayesian optimization

Bayesian optimizers use light weight models as surrogates for the target model — surrogates that can be evaluated quickly to recommend likely optimal hyperparameter values — and update those models using results from real trials.

Run `train` with Guild’s built-in Bayesian optimizer, which uses Gaussian processes. As with the earlier random search, we use Python's slice function to specify the range for `x` over which to search.

**NOTE:** The argument `bayesian` in the command is an alias for `gp`, which is a Bayesian optimizer that uses Gaussian processes. Guild supports three Bayesian optimizers: gp, forest and gbrt.

In [59]:
_ = guild.run(train, x=slice(-2.0,2.0), _optimizer="bayesian", _max_trials=10)

Running train (noise=0.1, x=1.5152199989091168):
loss: -0.024619
Running train (noise=0.1, x=-2.0):
loss: 0.039181
Running train (noise=0.1, x=2.0):
loss: -0.005486
Running train (noise=0.1, x=2.0):
loss: -0.118952
Running train (noise=0.1, x=2.0):
loss: -0.094768
Running train (noise=0.1, x=0.28882433942547214):
loss: 1.044828
Running train (noise=0.1, x=1.784311230545803):
loss: -0.035496
Running train (noise=0.1, x=-1.3192316849061574):
loss: -0.025329
Running train (noise=0.1, x=-0.8729053512489384):
loss: 0.364873
Running train (noise=0.1, x=-1.6124201016843505):
loss: 0.072872


View the top 10 results:

In [60]:
guild.runs().compare().sort_values(by="loss")[:10]

,run,operation,started,time,status,label,noise,x,step,loss
17,80a79058,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.300000,0,-0.854539
16,80a79059,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.200000,0,-0.738916
15,80a7905a,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.100000,0,-0.578615
18,80a79057,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.400000,0,-0.532630
19,80a79056,train(),2019-05-20 06:43:36,0:00:00,completed,,0.1,-0.500000,0,-0.449091
11,87f6c573,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,-0.559407,0,-0.317656
12,87f6c572,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,-0.019495,0,-0.316245
6,94f2f32a,train(),2019-05-20 06:44:10,0:00:00,completed,gp,0.1,2.000000,0,-0.118952
5,94f2f32b,train(),2019-05-20 06:44:10,0:00:00,completed,gp,0.1,2.000000,0,-0.094768
13,87f6c571,train(),2019-05-20 06:43:48,0:00:00,completed,random,0.1,-0.015538,0,-0.054769


## Next steps

This Notebook covers basic functionality provided by the `guild.ipy` module. The module lets you run functions and capture results as unique runs. You can view and compare run results using various module functions.

Generated runs can be further managed using the Guild command line interface. For more information, refer to [Guild AI Quick Start](https://guild.ai/docs/start/).